In [2]:
import os
import sys
import time
import networkx as nx
import scipy.io as sio
import scipy.sparse as sp
from scipy.spatial import distance_matrix
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.lines import Line2D
from matplotlib.animation import FuncAnimation

import pickle

%matplotlib inline

import re
numeric_const_pattern = r"""
     [-+]? 
     (?:
         (?: \d* \. \d+ )
         |
         (?: \d+ \.? )
     )
     (?: [Ee] [+-]? \d+ ) ?
     """
rx = re.compile(numeric_const_pattern, re.VERBOSE)

from IPython.display import clear_output
from IPython.display import Video

from scipy.stats.stats import pearsonr
import subprocess
from datetime import datetime


/tmp/ipykernel_3741080/1506328072.py:36: DeprecationWarning: Please import `pearsonr` from the `scipy.stats` namespace; the `scipy.stats.stats` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.stats.stats import pearsonr


In [3]:
# !sleep 3h;

In [4]:
logdir="bash"
output_dir = "log"
config_file = "zinc_gps_config_cos.csv"

df_all = pd.read_csv(os.path.join(logdir, config_file),index_col=0) 
# df = df.sort_values(by=["val_mae"])
df_all = df_all.reset_index(drop=True)
df_all

,opt,beta1,beta2,seed,epoch,loss,val_mae,test_mae,lr,env,optimizer,learning_rate
0,AMSGrad,0.9,0.900,seed10,1,0.688913,0.513300,0.536601,0.0001,zinc,AMSGrad,0.00050
1,AMSGrad+,0.9,0.999,seed10,1,0.688913,0.513300,0.536601,0.0001,zinc,AMSGrad+,0.00050
2,Lion,0.9,0.999,seed10,1,0.663569,0.523181,0.536426,0.0001,zinc,Lion,0.00025
3,Adam,0.9,0.999,seed10,1,0.692272,0.531897,0.551542,0.0001,zinc,Adam,0.00050
4,PIDAOSI,0.9,0.999,seed10,1,0.692272,0.531897,0.551542,0.0001,zinc,PIDAOSI,0.00050
5,AdaShift,0.9,0.999,seed10,1,0.692272,0.531897,0.551542,0.0001,zinc,AdaShift,0.00050
6,RMSprop,0.9,0.999,seed10,1,0.692272,0.531897,0.551542,0.0001,zinc,RMSprop,0.00050
7,Adopt,0.9,0.999,seed10,1,0.712638,0.533868,0.563475,0.0001,zinc,Adopt,0.00050
8,AdaGrad,0.9,0.999,seed10,1,0.667221,0.561272,0.582700,0.0010,zinc,AdaGrad,0.00250
9,AdaBelief,0.9,0.999,seed10,1,0.683830,0.563764,0.603262,0.0001,zinc,AdaBelief,0.00050


In [5]:
df=df_all.iloc[[33]]
# df=df_all.iloc[df_all.index >= 30] #.iloc[[6]]
df

,opt,beta1,beta2,seed,epoch,loss,val_mae,test_mae,lr,env,optimizer,learning_rate
33,Adam42,0.9,0.999,seed10,1,0.697401,0.549562,0.583326,0.0001,zinc,Adam42,0.0005


In [6]:
env = "zinc"
epochs = 1000
seed = "51"
atthead = "multihead"
db=-20
batch_size=128
# scheduler = 0 # constant learning rate no scheduled decay
scheduler = 0 # cosine learning rate 
max_threads = 12

In [ ]:
num_gpus=1
running_procs = []

for i, row in df.iterrows():
    opt = row["opt"]
    beta1 = row["beta1"]
    beta2 = row["beta2"]
    lr = row["lr"]

    env_copy = os.environ.copy()
    if num_gpus > 0:
        gpu_id = i % num_gpus  # round-robin assignment
        env_copy["CUDA_VISIBLE_DEVICES"] = "0" # str(gpu_id) # 
    else:
        env_copy["CUDA_VISIBLE_DEVICES"] = ""

    
    outname = f"{env}_epochs{epochs}_seed{seed}_{atthead}_{opt}_{beta2}_{lr}.txt"
    log_path = f"{output_dir}/{outname}"

    cmd = [
        "python3.10", "graph_gps.py",
        f"--seed={seed}",
        f"--learning_rate={lr}",
        f"--epochs={epochs}",
        f"--optimizer={opt}",
        f"--beta_1={beta1}",
        f"--beta_2={beta2}",
        f"--db={db}",
        f"--attn_type={atthead}",
        f"--batch_size={batch_size}",
        f"--scheduler={scheduler}",
    ]

    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{timestamp}][Submitting]: {' '.join(cmd)} > {log_path}")

    with open(log_path, "w") as outfile:
        proc = subprocess.Popen(
            cmd, stdout=outfile, stderr=outfile, env=env_copy, 
            preexec_fn=os.setsid,
        )
        running_procs.append(proc)

    # Limit number of concurrent jobs
    while len(running_procs) >= max_threads:
        for p in running_procs:
            if p.poll() is not None:  # finished
                running_procs.remove(p)
        time.sleep(30)  # wait a bit before checking again

# Wait for remaining jobs to finish
# for p in running_procs:
#     p.wait()
